In [1]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
#LOADING SAVED MODEL
import joblib


# Load the saved models
models = joblib.load('blended_models.pkl')

xgb_loaded = models['xgboost']
tabpfn_loaded = models['tabpfn']

In [3]:
#PREPARE NEW DATA FOR INFERENCE
# Example: New real-world customer data (same features as training set!)
import pandas as pd
import pandas as pd

# Data for 3 different customers
new_data = pd.DataFrame([
    {
        'Geography_Germany': 1,
        'Gender': 1,
        'Age': 35,
        'Tenure': 3,
        'Balance': 50000,
        'NumOfProducts': 1,
        'HasCrCard': 1,
        'IsActiveMember': 0
    },
    {
        'Geography_Germany': 0,
        'Gender': 0,
        'Age': 42,
        'Tenure': 6,
        'Balance': 120000,
        'NumOfProducts': 2,
        'HasCrCard': 0,
        'IsActiveMember': 1
    },
      {
        'Geography_Germany': 1,
        'Gender': 0,               # Female
        'Age': 57,
        'Tenure': 2,
        'Balance': 0,
        'NumOfProducts': 1,
        'HasCrCard': 0,
        'IsActiveMember': 0
    }
])

new_data

,Geography_Germany,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember
0,1,1,35,3,50000,1,1,0
1,0,0,42,6,120000,2,0,1
2,1,0,57,2,0,1,0,0


In [4]:
#MAKE BLENDED PREDICTIONS
threshold = 0.3  # same threshold as training

# Predict probabilities for all 3 customers
tabpfn_probs = tabpfn_loaded.predict_proba(new_data)[:, 1]
xgb_probs = xgb_loaded.predict_proba(new_data)[:, 1]

# Blend the predictions
blended_probs = 0.3 * xgb_probs + 0.7 * tabpfn_probs

# Apply threshold to get final predictions
final_predictions = (blended_probs >= threshold).astype(int)

# Display results
for i in range(len(new_data)):
    print(f"Customer {i+1}")
    print(f"🔍 Churn Probability: {blended_probs[i]:.4f}")
    print("🧾 Final Prediction:", "Churn" if final_predictions[i] == 1 else "Not Churn")
    print("-" * 40)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but TabPFNClassifier was fitted without feature names
  warnings.warn(


Customer 1
🔍 Churn Probability: 0.7808
🧾 Final Prediction: Churn
----------------------------------------
Customer 2
🔍 Churn Probability: 0.7517
🧾 Final Prediction: Churn
----------------------------------------
Customer 3
🔍 Churn Probability: 0.2163
🧾 Final Prediction: Not Churn
----------------------------------------
